# **Week 6 Code Explanation**


## **Step 0:** Preprocessing the Grap


In [11]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#IMPORTING DATABASE
edges_filename = "edges.csv"
nodes_filename = "nodes.csv"

df_edges = pd.read_csv(edges_filename)
df_nodes = pd.read_csv(nodes_filename)

# Create the graph using the edges CSV
G = nx.from_pandas_edgelist(df_edges, '# source', ' target')

# Ensure the graph is undirected and unweighted
G = nx.to_undirected(G)

# Remove self-loops
if nx.is_frozen(G):
    # Unfreeze the graph if it's frozen
    G = nx.Graph(G)
G.remove_edges_from(nx.selfloop_edges(G))

# Keep only the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

## **Step 1:** Implement Community Detection Techniques

### **A** Bridge Removal

#### Removing Bridges and Identify Components:
First find bridges, remove them, and then find the connected components of the resulting graph.

In [14]:
G_no_bridges = G.copy()
bridges = list(nx.bridges(G_no_bridges))
G_no_bridges.remove_edges_from(bridges)
components = [G_no_bridges.subgraph(c).copy() for c in nx.connected_components(G_no_bridges)]

#### Compute and Select Highest Modularity:
Calculate modularity for each component and select the one with the highest modularity.

In [15]:
modularity_scores = []
for comp in components:
    community = list(comp.nodes)
    other_nodes = set(G.nodes()) - set(community)
    partition = [community, list(other_nodes)]
    modularity_score = nx.community.modularity(G, partition)
    modularity_scores.append(modularity_score)

max_modularity_index = modularity_scores.index(max(modularity_scores))
best_partition_bridge_removal = components[max_modularity_index]

### **B** Modularity Optimization

Use the Louvain method for modularity optimization

In [20]:
import community as community_louvain

partition_modularity_optimization = community_louvain.best_partition(G)


AttributeError: module 'community' has no attribute 'best_partition'